## The challenge:

In this competition you’ll predict what types of trees there are in an area based on various geographic features.

The competition datasets comes from a study conducted in four wilderness areas within the beautiful Roosevelt National Forest of northern Colorado. These areas represent forests with very little human disturbances – the existing forest cover types there are more a result of ecological processes rather than forest management practices.

The data is in raw form and contains categorical data such as wilderness areas and soil type.

## Import Packages

In [1]:
DATA_DIR = '/kaggle/input/learn-together'
DATA_DIR = 'data'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pybr/kaggle-fores

data/train.csv
data/test.csv
data/sample_submission.csv
data/sample_submission.csv.zip
data/input
data/test.csv.zip
data/train.csv.zip


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/i

In [3]:
print(tf.__version__)

1.14.0


In [4]:
def report(y_true, y_pred):
    print('Accuracy: %s' % accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

## Load Dataset

In [5]:
train_df=pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df=pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [6]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [7]:
test_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [8]:
print("shape training csv: %s" % str(train_df.shape)) 
print("shape test csv: %s" % str(test_df.shape)) 

shape training csv: (15120, 56)
shape test csv: (565892, 55)


## Delete Ids
**Let's delete the Id column in the training set but store it for the test set before deleting**

In [9]:
train_df = train_df.drop(["Id"], axis = 1)

test_ids = test_df["Id"]
test_df = test_df.drop(["Id"], axis = 1)

In [10]:
train_df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ss = StandardScaler()
mm = MinMaxScaler()
numerical_df = train_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']]

numerical_df_test = test_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']]



In [12]:
ss.fit(pd.concat([numerical_df, numerical_df_test]))
X_ss = ss.transform(numerical_df)
X_ss_test = ss.transform(numerical_df_test)

In [13]:
def categorical_features(train_df, test_df):
    size = len(train_df)
    size_test = len(test_df)
    columns_to_keep = []
    for column in train_df.columns:
        if 'Soil' in column or 'Wilderness' in column:
            frequency = train_df[column].sum() / size
            if frequency >= 0.01:
                columns_to_keep.append(column)
    print('Columns keeped %s' % columns_to_keep)
    return np.array(train_df[columns_to_keep]), np.array(test_df[columns_to_keep])

In [14]:
categorical, categorical_test = categorical_features(train_df, test_df)

Columns keeped ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type17', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']


In [15]:
categorical[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0]])

In [16]:
X = np.hstack((X_ss, categorical))

In [17]:
X_test = np.hstack((X_ss_test, categorical_test))

In [18]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()
y = binarizer.fit_transform(train_df['Cover_Type'])

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [20]:
def model1():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [21]:
def model2():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [22]:
def model3():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [23]:
def compile_and_train(model, epochs=50, verbose=0):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)
    print(model.metrics_names)
    print(model.evaluate(X_train, y_train))
    print(model.evaluate(X_val, y_val))

In [24]:
for model in [model1(), model2(), model3()]:
    compile_and_train(model)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
['loss', 'acc']
13608/13608 [==============================] - 0s 25us/sample - loss: 0.5222 - acc: 0.7834
[0.5222126395123485, 0.78336275]
1512/1512 [==============================] - 0s 28us/sample - loss: 0.5607 - acc: 0.7758
[0.5607167203274984, 0.7757937]
['loss', 'acc']
13608/13608 [==============================] - 0s 26us/sample - loss: 0.4488 - acc: 0.8207
[0.44877723915867074, 0.82069373]
1512/1512 [==============================] - 0s 29us/sample - loss: 0.4956 - acc: 0.7956
[0.4956313802136315, 0.7956349]
['loss', 'acc']
13608/13608 [==============================] - 0s 29us/sample - loss: 0.5163 - acc: 0.7787
[0.5163364883803536, 0.77865964]
1512/1512 [==============================] - 0s 29us/sample - loss: 0.5542 - acc: 0.7652
[0.5542449656302336, 0.76521164]


## Final training

### Callbacks

In [44]:
tensorboard_logs = TensorBoard(log_dir='./logs', histogram_freq=1,
                               write_graph=True, write_images=True,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join('./keras_models', 
                                        'modelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_acc', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=60, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=40,
                              verbose=1, min_delta=1e-2, mode='min')

### Model

### Training

In [26]:
model = model2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=500,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          batch_size=378)


Train on 13608 samples, validate on 1512 samples
Epoch 1/500
13608/13608 [==============================] - 2s 118us/sample - loss: 1.2634 - acc: 0.4727 - val_loss: 0.8028 - val_acc: 0.6786
Epoch 2/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.9073 - acc: 0.6141 - val_loss: 0.7187 - val_acc: 0.6978
Epoch 3/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.8199 - acc: 0.6521 - val_loss: 0.6888 - val_acc: 0.6978
Epoch 4/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.7819 - acc: 0.6704 - val_loss: 0.6530 - val_acc: 0.7163
Epoch 5/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.7459 - acc: 0.6843 - val_loss: 0.6445 - val_acc: 0.7183
Epoch 6/500
13608/13608 [==============================] - 1s 71us/sample - loss: 0.7294 - acc: 0.6981 - val_loss: 0.6225 - val_acc: 0.7374
Epoch 7/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.7133 - acc: 0.7016 - val_l

13608/13608 [==============================] - 1s 65us/sample - loss: 0.5538 - acc: 0.7726 - val_loss: 0.5058 - val_acc: 0.7890
Epoch 57/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.5557 - acc: 0.7695 - val_loss: 0.4988 - val_acc: 0.7956
Epoch 58/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.5471 - acc: 0.7732 - val_loss: 0.4996 - val_acc: 0.7983
Epoch 59/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.5442 - acc: 0.7738 - val_loss: 0.4934 - val_acc: 0.7943
Epoch 60/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5499 - acc: 0.7743 - val_loss: 0.4976 - val_acc: 0.7976
Epoch 61/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5420 - acc: 0.7782 - val_loss: 0.5036 - val_acc: 0.7923
Epoch 62/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5497 - acc: 0.7730 - val_loss: 0.4927 - val_acc: 0.7963
Epoch 63/500
13608/13608 [

Epoch 113/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5087 - acc: 0.7905 - val_loss: 0.4599 - val_acc: 0.8155
Epoch 114/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5152 - acc: 0.7891 - val_loss: 0.4679 - val_acc: 0.8115
Epoch 115/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5129 - acc: 0.7898 - val_loss: 0.4602 - val_acc: 0.8148
Epoch 116/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5083 - acc: 0.7922 - val_loss: 0.4582 - val_acc: 0.8221
Epoch 117/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5138 - acc: 0.7887 - val_loss: 0.4607 - val_acc: 0.8148
Epoch 118/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5091 - acc: 0.7911 - val_loss: 0.4681 - val_acc: 0.8082
Epoch 119/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5153 - acc: 0.7875 - val_loss: 0.4621 - val_acc: 0.8168
Epoch 

Epoch 168/500
13608/13608 [==============================] - 1s 64us/sample - loss: 0.4856 - acc: 0.8029 - val_loss: 0.4490 - val_acc: 0.8175
Epoch 169/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.4864 - acc: 0.8020 - val_loss: 0.4453 - val_acc: 0.8208
Epoch 170/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.4813 - acc: 0.8034 - val_loss: 0.4554 - val_acc: 0.8102
Epoch 171/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.4768 - acc: 0.8040 - val_loss: 0.4480 - val_acc: 0.8221
Epoch 172/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.4801 - acc: 0.8042 - val_loss: 0.4469 - val_acc: 0.8168
Epoch 173/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.4829 - acc: 0.8063 - val_loss: 0.4476 - val_acc: 0.8221
Epoch 174/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.4857 - acc: 0.8024 - val_loss: 0.4456 - val_acc: 0.8175
Epoch 

In [27]:
y_pred = model.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8234126984126984
              precision    recall  f1-score   support

           1       0.73      0.75      0.74       203
           2       0.75      0.57      0.65       209
           3       0.81      0.71      0.75       217
           4       0.93      0.97      0.95       234
           5       0.87      0.93      0.90       224
           6       0.71      0.83      0.77       200
           7       0.92      0.97      0.94       225

    accuracy                           0.82      1512
   macro avg       0.82      0.82      0.81      1512
weighted avg       0.82      0.82      0.82      1512

[[152  29   0   0   4   1  17]
 [ 49 120   9   0  21   7   3]
 [  0   3 153   9   4  48   0]
 [  0   0   2 228   0   4   0]
 [  1   6   2   0 208   7   0]
 [  0   1  23   9   1 166   0]
 [  7   0   0   0   0   0 218]]


In [28]:
model = model2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=500,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          batch_size=378)

Train on 15120 samples, validate on 1512 samples
Epoch 1/500
15120/15120 [==============================] - 1s 81us/sample - loss: 1.2356 - acc: 0.4836 - val_loss: 0.7847 - val_acc: 0.6905
Epoch 2/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.8781 - acc: 0.6325 - val_loss: 0.7012 - val_acc: 0.7183
Epoch 3/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.8051 - acc: 0.6604 - val_loss: 0.6649 - val_acc: 0.7295
Epoch 4/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.7679 - acc: 0.6752 - val_loss: 0.6376 - val_acc: 0.7242
Epoch 5/500
15120/15120 [==============================] - 1s 65us/sample - loss: 0.7404 - acc: 0.6876 - val_loss: 0.6177 - val_acc: 0.7381
Epoch 6/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.7170 - acc: 0.7032 - val_loss: 0.6035 - val_acc: 0.7526
Epoch 7/500
15120/15120 [==============================] - 1s 65us/sample - loss: 0.7092 - acc: 0.7026 - val_lo

Epoch 59/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5377 - acc: 0.7754 - val_loss: 0.4228 - val_acc: 0.8175
Epoch 60/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5393 - acc: 0.7768 - val_loss: 0.4211 - val_acc: 0.8254
Epoch 61/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5466 - acc: 0.7752 - val_loss: 0.4226 - val_acc: 0.8294
Epoch 62/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5381 - acc: 0.7761 - val_loss: 0.4239 - val_acc: 0.8287
Epoch 63/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5443 - acc: 0.7754 - val_loss: 0.4154 - val_acc: 0.8208
Epoch 64/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5393 - acc: 0.7732 - val_loss: 0.4169 - val_acc: 0.8247
Epoch 65/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5346 - acc: 0.7810 - val_loss: 0.4137 - val_acc: 0.8294
Epoch 66/500


Epoch 117/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5083 - acc: 0.7894 - val_loss: 0.3738 - val_acc: 0.8439
Epoch 118/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4990 - acc: 0.7899 - val_loss: 0.3723 - val_acc: 0.8472
Epoch 119/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.5005 - acc: 0.7902 - val_loss: 0.3738 - val_acc: 0.8519
Epoch 120/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4990 - acc: 0.7953 - val_loss: 0.3727 - val_acc: 0.8433
Epoch 121/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4931 - acc: 0.7979 - val_loss: 0.3700 - val_acc: 0.8452
Epoch 122/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4988 - acc: 0.7962 - val_loss: 0.3680 - val_acc: 0.8466
Epoch 123/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4967 - acc: 0.7943 - val_loss: 0.3640 - val_acc: 0.8519
Epoch 

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4853 - acc: 0.8005 - val_loss: 0.3491 - val_acc: 0.8598
Epoch 174/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4791 - acc: 0.8041 - val_loss: 0.3494 - val_acc: 0.8552
Epoch 175/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4783 - acc: 0.8093 - val_loss: 0.3456 - val_acc: 0.8571
Epoch 176/500
14816/15120 [============================>.] - ETA: 0s - loss: 0.4865 - acc: 0.7999
Epoch 00176: ReduceLROnPlateau reducing learning rate to 0.0005120000336319208.
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4861 - acc: 0.7999 - val_loss: 0.3443 - val_acc: 0.8677
Epoch 177/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4766 - acc: 0.8036 - val_loss: 0.3490 - val_acc: 0.8624
Epoch 178/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4761 - acc: 0.8031 - val_loss: 0.3410 - val_acc: 0.867

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4521 - acc: 0.8138 - val_loss: 0.3232 - val_acc: 0.8684
Epoch 228/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4534 - acc: 0.8161 - val_loss: 0.3271 - val_acc: 0.8677
Epoch 229/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4567 - acc: 0.8137 - val_loss: 0.3284 - val_acc: 0.8710
Epoch 230/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4560 - acc: 0.8124 - val_loss: 0.3281 - val_acc: 0.8710
Epoch 231/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4593 - acc: 0.8156 - val_loss: 0.3283 - val_acc: 0.8657
Epoch 232/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4594 - acc: 0.8125 - val_loss: 0.3300 - val_acc: 0.8664
Epoch 233/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4597 - acc: 0.8138 - val_loss: 0.3273 - val_acc: 0.8737
Epoch 234/500
15120/

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4424 - acc: 0.8202 - val_loss: 0.3177 - val_acc: 0.8730
Epoch 283/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4459 - acc: 0.8188 - val_loss: 0.3168 - val_acc: 0.8790
Epoch 284/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4412 - acc: 0.8189 - val_loss: 0.3137 - val_acc: 0.8796
Epoch 285/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4459 - acc: 0.8185 - val_loss: 0.3139 - val_acc: 0.8763
Epoch 286/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4442 - acc: 0.8218 - val_loss: 0.3167 - val_acc: 0.8776
Epoch 287/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4517 - acc: 0.8144 - val_loss: 0.3148 - val_acc: 0.8743
Epoch 288/500
15120/15120 [==============================] - 1s 65us/sample - loss: 0.4448 - acc: 0.8182 - val_loss: 0.3146 - val_acc: 0.8757
Epoch 289/500
15120/

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4369 - acc: 0.8227 - val_loss: 0.3089 - val_acc: 0.8829
Epoch 338/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4406 - acc: 0.8210 - val_loss: 0.3112 - val_acc: 0.8763
Epoch 339/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4438 - acc: 0.8187 - val_loss: 0.3099 - val_acc: 0.8770
Epoch 340/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4456 - acc: 0.8198 - val_loss: 0.3116 - val_acc: 0.8757
Epoch 341/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4448 - acc: 0.8225 - val_loss: 0.3132 - val_acc: 0.8796
Epoch 342/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4427 - acc: 0.8228 - val_loss: 0.3102 - val_acc: 0.8783
Epoch 343/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4390 - acc: 0.8228 - val_loss: 0.3078 - val_acc: 0.8763
Epoch 344/500
14848/

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4361 - acc: 0.8210 - val_loss: 0.3054 - val_acc: 0.8856
Epoch 392/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4345 - acc: 0.8238 - val_loss: 0.3028 - val_acc: 0.8843
Epoch 393/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4269 - acc: 0.8265 - val_loss: 0.3022 - val_acc: 0.8862
Epoch 394/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4376 - acc: 0.8257 - val_loss: 0.3027 - val_acc: 0.8849
Epoch 395/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4343 - acc: 0.8269 - val_loss: 0.3040 - val_acc: 0.8803
Epoch 396/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4303 - acc: 0.8251 - val_loss: 0.3028 - val_acc: 0.8816
Epoch 397/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4285 - acc: 0.8290 - val_loss: 0.3020 - val_acc: 0.8836
Epoch 398/500
15120/

15120/15120 [==============================] - 1s 64us/sample - loss: 0.4372 - acc: 0.8235 - val_loss: 0.3013 - val_acc: 0.8829
Epoch 447/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4338 - acc: 0.8267 - val_loss: 0.3013 - val_acc: 0.8849
Epoch 448/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4312 - acc: 0.8269 - val_loss: 0.3023 - val_acc: 0.8836
Epoch 449/500
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4314 - acc: 0.8262 - val_loss: 0.3016 - val_acc: 0.8836
Epoch 450/500
14912/15120 [============================>.] - ETA: 0s - loss: 0.4315 - acc: 0.8234
Epoch 00450: ReduceLROnPlateau reducing learning rate to 3.518437442835421e-05.
15120/15120 [==============================] - 1s 64us/sample - loss: 0.4313 - acc: 0.8235 - val_loss: 0.3015 - val_acc: 0.8862
Epoch 451/500
15120/15120 [==============================] - 1s 65us/sample - loss: 0.4349 - acc: 0.8239 - val_loss: 0.3017 - val_acc: 0.883

In [31]:
y_pred = model.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8835978835978836
              precision    recall  f1-score   support

           1       0.81      0.79      0.80       203
           2       0.84      0.71      0.77       209
           3       0.87      0.83      0.85       217
           4       0.94      0.98      0.96       234
           5       0.92      0.98      0.95       224
           6       0.84      0.88      0.86       200
           7       0.94      0.98      0.96       225

    accuracy                           0.88      1512
   macro avg       0.88      0.88      0.88      1512
weighted avg       0.88      0.88      0.88      1512

[[161  28   0   0   2   0  12]
 [ 32 149   8   0  14   4   2]
 [  0   0 181  11   1  24   0]
 [  0   0   1 230   0   3   0]
 [  0   1   1   0 219   3   0]
 [  0   0  18   4   2 176   0]
 [  5   0   0   0   0   0 220]]


In [49]:
modelw = model2()
modelw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
modelw.fit(X, y, epochs=1000,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          class_weight = {0:2, 1:2, 2:1, 3:1, 4:1, 5:1, 6:1},
          batch_size=378)

Train on 15120 samples, validate on 1512 samples
Epoch 1/1000
15120/15120 [==============================] - 1s 45us/sample - loss: 2.2818 - acc: 0.2356 - val_loss: 1.4775 - val_acc: 0.3261
Epoch 2/1000
15120/15120 [==============================] - 0s 16us/sample - loss: 1.7370 - acc: 0.3869 - val_loss: 1.0573 - val_acc: 0.5483
Epoch 3/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.4664 - acc: 0.4837 - val_loss: 0.8952 - val_acc: 0.6389
Epoch 4/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.3158 - acc: 0.5519 - val_loss: 0.8197 - val_acc: 0.6779
Epoch 5/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.2381 - acc: 0.5859 - val_loss: 0.7796 - val_acc: 0.6806
Epoch 6/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.1898 - acc: 0.6034 - val_loss: 0.7678 - val_acc: 0.6806
Epoch 7/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.1671 - acc: 0.6101 - val_

Epoch 59/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.8411 - acc: 0.7399 - val_loss: 0.5230 - val_acc: 0.7917
Epoch 60/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8321 - acc: 0.7401 - val_loss: 0.5182 - val_acc: 0.8009
Epoch 61/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8237 - acc: 0.7419 - val_loss: 0.5162 - val_acc: 0.7963
Epoch 62/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8392 - acc: 0.7421 - val_loss: 0.5163 - val_acc: 0.7970
Epoch 63/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8212 - acc: 0.7440 - val_loss: 0.5221 - val_acc: 0.7917
Epoch 64/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.8283 - acc: 0.7455 - val_loss: 0.5172 - val_acc: 0.7956
Epoch 65/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8319 - acc: 0.7429 - val_loss: 0.5151 - val_acc: 0.7890
Epoch 66/1000

Epoch 117/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7538 - acc: 0.7676 - val_loss: 0.4521 - val_acc: 0.8194
Epoch 118/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7656 - acc: 0.7688 - val_loss: 0.4564 - val_acc: 0.8214
Epoch 119/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7687 - acc: 0.7685 - val_loss: 0.4589 - val_acc: 0.8208
Epoch 120/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7576 - acc: 0.7661 - val_loss: 0.4565 - val_acc: 0.8128
Epoch 121/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7532 - acc: 0.7690 - val_loss: 0.4598 - val_acc: 0.8115
Epoch 122/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7486 - acc: 0.7735 - val_loss: 0.4520 - val_acc: 0.8201
Epoch 123/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7557 - acc: 0.7690 - val_loss: 0.4580 - val_acc: 0.8161
Epoch 

Epoch 175/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7184 - acc: 0.7800 - val_loss: 0.4215 - val_acc: 0.8188
Epoch 176/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7170 - acc: 0.7792 - val_loss: 0.4155 - val_acc: 0.8214
Epoch 177/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7176 - acc: 0.7824 - val_loss: 0.4115 - val_acc: 0.8267
Epoch 178/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7117 - acc: 0.7858 - val_loss: 0.4167 - val_acc: 0.8280
Epoch 179/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7187 - acc: 0.7798 - val_loss: 0.4188 - val_acc: 0.8214
Epoch 180/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7172 - acc: 0.7845 - val_loss: 0.4108 - val_acc: 0.8300
Epoch 181/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7115 - acc: 0.7826 - val_loss: 0.4115 - val_acc: 0.8320
Epoch 

Epoch 233/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6965 - acc: 0.7896 - val_loss: 0.3998 - val_acc: 0.8320
Epoch 234/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.7041 - acc: 0.7906 - val_loss: 0.3979 - val_acc: 0.8373
Epoch 235/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6877 - acc: 0.7969 - val_loss: 0.3855 - val_acc: 0.8479
Epoch 236/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6913 - acc: 0.7937 - val_loss: 0.3888 - val_acc: 0.8485
Epoch 237/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6915 - acc: 0.7903 - val_loss: 0.3916 - val_acc: 0.8439
Epoch 238/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6913 - acc: 0.7899 - val_loss: 0.3892 - val_acc: 0.8413
Epoch 239/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6931 - acc: 0.7898 - val_loss: 0.3901 - val_acc: 0.8433
Epoch 

...

15120/15120 [==============================] - 0s 8us/sample - loss: 0.6395 - acc: 0.8089 - val_loss: 0.3432 - val_acc: 0.8657
Epoch 570/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6504 - acc: 0.8061 - val_loss: 0.3448 - val_acc: 0.8664
Epoch 571/1000
14364/15120 [===========================>..] - ETA: 0s - loss: 0.6439 - acc: 0.8063
Epoch 00571: ReduceLROnPlateau reducing learning rate to 0.00020971521735191345.
15120/15120 [==============================] - 0s 9us/sample - loss: 0.6431 - acc: 0.8067 - val_loss: 0.3437 - val_acc: 0.8644
Epoch 572/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6338 - acc: 0.8089 - val_loss: 0.3449 - val_acc: 0.8651
Epoch 573/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6403 - acc: 0.8059 - val_loss: 0.3440 - val_acc: 0.8651
Epoch 574/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6358 - acc: 0.8086 - val_loss: 0.3434 - val_acc: 0.864

15120/15120 [==============================] - 0s 8us/sample - loss: 0.6445 - acc: 0.8056 - val_loss: 0.3410 - val_acc: 0.8657
Epoch 625/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6340 - acc: 0.8118 - val_loss: 0.3398 - val_acc: 0.8664
Epoch 626/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6338 - acc: 0.8101 - val_loss: 0.3394 - val_acc: 0.8671
Epoch 627/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6317 - acc: 0.8106 - val_loss: 0.3390 - val_acc: 0.8704
Epoch 628/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6356 - acc: 0.8082 - val_loss: 0.3392 - val_acc: 0.8644
Epoch 629/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6350 - acc: 0.8050 - val_loss: 0.3409 - val_acc: 0.8651
Epoch 630/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6407 - acc: 0.8076 - val_loss: 0.3419 - val_acc: 0.8671
Epoch 631/1000
15120/

15120/15120 [==============================] - 0s 8us/sample - loss: 0.6394 - acc: 0.8081 - val_loss: 0.3369 - val_acc: 0.8684
Epoch 681/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6418 - acc: 0.8058 - val_loss: 0.3396 - val_acc: 0.8717
Epoch 682/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6325 - acc: 0.8088 - val_loss: 0.3393 - val_acc: 0.8677
Epoch 683/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6306 - acc: 0.8080 - val_loss: 0.3374 - val_acc: 0.8697
Epoch 684/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6263 - acc: 0.8089 - val_loss: 0.3366 - val_acc: 0.8697
Epoch 685/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6300 - acc: 0.8134 - val_loss: 0.3362 - val_acc: 0.8710
Epoch 686/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6348 - acc: 0.8109 - val_loss: 0.3351 - val_acc: 0.8704
Epoch 687/1000
15120/

15120/15120 [==============================] - 0s 8us/sample - loss: 0.6345 - acc: 0.8081 - val_loss: 0.3361 - val_acc: 0.8684
Epoch 737/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6330 - acc: 0.8095 - val_loss: 0.3360 - val_acc: 0.8684
Epoch 738/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6267 - acc: 0.8121 - val_loss: 0.3352 - val_acc: 0.8704
Epoch 739/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6312 - acc: 0.8110 - val_loss: 0.3355 - val_acc: 0.8697
Epoch 740/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6357 - acc: 0.8110 - val_loss: 0.3359 - val_acc: 0.8671
Epoch 741/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6277 - acc: 0.8117 - val_loss: 0.3352 - val_acc: 0.8697
Epoch 742/1000
 7182/15120 [=============>................] - ETA: 0s - loss: 0.6318 - acc: 0.8165
Epoch 00742: ReduceLROnPlateau reducing learning rate to 8.58993

15120/15120 [==============================] - 0s 9us/sample - loss: 0.6237 - acc: 0.8156 - val_loss: 0.3352 - val_acc: 0.8671
Epoch 792/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6363 - acc: 0.8091 - val_loss: 0.3358 - val_acc: 0.8690
Epoch 793/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6275 - acc: 0.8064 - val_loss: 0.3365 - val_acc: 0.8684
Epoch 794/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6316 - acc: 0.8117 - val_loss: 0.3361 - val_acc: 0.8684
Epoch 795/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6244 - acc: 0.8121 - val_loss: 0.3354 - val_acc: 0.8704
Epoch 796/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6338 - acc: 0.8079 - val_loss: 0.3367 - val_acc: 0.8664
Epoch 797/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6276 - acc: 0.8146 - val_loss: 0.3360 - val_acc: 0.8684
Epoch 798/1000
15120/

In [50]:
y_pred = modelw.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.871031746031746
              precision    recall  f1-score   support

           1       0.86      0.76      0.81       203
           2       0.77      0.84      0.81       209
           3       0.84      0.75      0.80       217
           4       0.94      1.00      0.96       234
           5       0.93      0.93      0.93       224
           6       0.78      0.83      0.80       200
           7       0.97      0.96      0.96       225

    accuracy                           0.87      1512
   macro avg       0.87      0.87      0.87      1512
weighted avg       0.87      0.87      0.87      1512

[[155  39   0   0   3   0   6]
 [ 16 176   5   0   8   3   1]
 [  0   2 163   8   3  41   0]
 [  0   0   1 233   0   0   0]
 [  0  10   2   0 208   4   0]
 [  0   1  22   8   2 167   0]
 [ 10   0   0   0   0   0 215]]


## Predictions

In [51]:
test_pred = modelw.predict(X_test)

In [52]:
# Save test predictions to file
output = pd.DataFrame({'ID': test_ids,
                       'Cover_Type': binarizer.inverse_transform(test_pred)})
output.to_csv('submission_neural.csv', index=False)